In [283]:
%config IPCompleter.greedy=True

In [284]:
import networkx as nx
import matplotlib.pyplot as plt

## Function: Conll file parser and graph creater

In [285]:
def create_graph_conll(conll):
    """
    Conll file format: http://sdp.delph-in.net/2014/data.html
    """
    
    def add_edges(graph, edges, preds):
        for i in edges:
            _from = i[0]
            _to = preds[i[1] - 1]
            _label = i[2]
            graph.add_edge(_from, _to, label=_label)
    
    graphs = []
    _graph = False
    _edges = []
    _preds = []
    
    for line in conll.readlines():
        # new line
        if line[0] == '#':
            
            # add edges
            if (_graph):
                add_edges(_graph, _edges, _preds)
                _edges = []
                _preds = []
                
            _graph = nx.DiGraph()
            _info = {}
            graphs.append([_graph, _info])
        
        # each line of sentence
        # id, form, lemma, pos, top, pred, [as many args as preds]
        elif len(line) > 1:
            _current_graph = graphs[len(graphs) - 1]
            _words = line.replace('\n', '').split('\t')
            _id = int(_words[0])
            _form = _words[1]
            _lemma = _words[2]
            _pos = _words[3]
            _top = _words[4] == '+'
            _pred = _words[5] == '+'
            _info[_id] = {
                'form': _form,
                'lemma': _lemma,
                'pos': _pos,
                'top': _top,
                'pred': _pred
            }
            
            # Storing predicates
            if _pred:
                _preds.append(_id)
                
            if _top:
                _graph.add_edge(_id, _id, label='TOP')
            
            _graph.add_node(_id)
            
            # Storing edges
            for i in range(6, len(_words[6:]) + 6):
                if _words[i] != '_':
                    _edges.append((_id, i-5, _words[i]))
    
    # cleanup on last sentence
    add_edges(_graph, _edges, _preds)
    return graphs

## Function: Full analysis

In [286]:
def dependencies(gold, test):
    """
    Precision, recall, F-score for dependencies
    """
    found = 0
    found_labeled = 0
    
    gold_size = 0
    gold_size_labeled = 0
    
    test_size = 0
    test_size_labeled = 0
    
    for a, b in zip(gold, test):
        
        # graphs
        g_gold = a[0].edges()
        g_gold_labeled = a[0].edges(data='label')
        g_test = b[0].edges()
        g_test_labeled = b[0].edges(data='label')
        
        # sizes
        gold_size += len(g_gold)
        gold_size_labeled += len(g_gold_labeled)
        test_size += len(g_test)
        test_size_labeled += len(g_test_labeled)
        
        for edge in g_test:
            if edge in g_gold:
                found += 1
                
        for edge in g_test_labeled:
            if edge in g_gold_labeled:
                found_labeled += 1
                
    p = found / test_size
    r = found / gold_size
    f = 2 * ((p * r) / (p + r))
    
    pl = found_labeled / test_size_labeled
    rl = found_labeled / gold_size_labeled
    fl = 2 * ((pl * rl) / (pl + rl))
    
    print('--------------------------------------')
    print('{:<30} {:<30} {:<30}'.format('precision', 'recall', 'f-score'))
    print('Unlabeled')
    print('{:<30} {:<30} {:<30}'.format(p, r, f))
    print('Labeled')
    print('{:<30} {:<30} {:<30}'.format(pl, rl, fl))

## Function: Get arguments

In [287]:
def get_args(conll):
    args = []
    for line in conll.readlines():
        if line[0] != '#' and len(line) > 1:
            _words = line.replace('\n', '').split('\t')
            for i in range(6, len(_words[6:]) + 6):
                if _words[i] != '_':
                    if _words[i] not in args:
                        args.append(_words[i])
    return args

## Reading in 2014 data

In [391]:
dm = create_graph_conll(open('2014/test/dm.sdp'))
pas = create_graph_conll(open('2014/test/pas.sdp'))
pcedt = create_graph_conll(open('2014/test/pcedt.sdp'))
_trial = create_graph_conll(open('2014/trial/dm.sdp'))

In [289]:
dm_args = get_args(open('2014/test/dm.sdp'))
pas_args = get_args(open('2014/test/pas.sdp'))
pcedt_args = get_args(open('2014/test/pcedt.sdp'))
print(len(dm_args))
print(len(pas_args))
print(len(pcedt_args))

35
41
62


# Data for entire graph

## Alpage

In [505]:
# closed
alpage_dm1_c = create_graph_conll(open('2014/submissions/alpage/closed.dm.1.sdp'))
alpage_dm2_c = create_graph_conll(open('2014/submissions/alpage/closed.dm.2.sdp'))
alpage_pas1_c = create_graph_conll(open('2014/submissions/alpage/closed.pas.1.sdp'))
alpage_pas2_c = create_graph_conll(open('2014/submissions/alpage/closed.pas.2.sdp'))
# alpage_pcedt1_c = create_graph_conll(open('2014/submissions/alpage/closed.pcedt.1.sdp'))
alpage_pcedt2_c = create_graph_conll(open('2014/submissions/alpage/closed.pcedt.2.sdp'))

# open
alpage_dm1_o = create_graph_conll(open('2014/submissions/alpage/open.dm.1.sdp'))
alpage_dm2_o = create_graph_conll(open('2014/submissions/alpage/open.dm.2.sdp'))
alpage_pas1_o = create_graph_conll(open('2014/submissions/alpage/open.pas.1.sdp'))
alpage_pas2_o = create_graph_conll(open('2014/submissions/alpage/open.pas.2.sdp'))
# alpage_pcedt1_o = create_graph_conll(open('2014/submissions/alpage/open.pcedt.1.sdp'))
alpage_pcedt2_o = create_graph_conll(open('2014/submissions/alpage/open.pcedt.2.sdp'))


print('closed')
print('\ndm1')
dependencies(dm, alpage_dm1_c)
print('\ndm2')
dependencies(dm, alpage_dm2_c)
print('\npas1')
dependencies(pas, alpage_pas1_c)
print('\npas2')
dependencies(pas, alpage_pas2_c)
# print('pcedt1')
# dependencies(pas, alpage_pcedt1_c)
print('\npcedt2')
dependencies(pcedt, alpage_pcedt2_c)

print('\nopen')
print('\ndm1')
dependencies(dm, alpage_dm1_o)
print('\ndm2')
dependencies(dm, alpage_dm2_o)
print('\npas1')
dependencies(pas, alpage_pas1_o)
print('\npas2')
dependencies(pas, alpage_pas2_o)
# print('pcedt1')
# dependencies(pas, alpage_pcedt1_o)
print('\npcedt2')
dependencies(pcedt, alpage_pcedt2_o)

closed

dm1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8206664004614225             0.7971813989570314             0.8087534432250449            
Labeled
0.7959536802875017             0.7731758824289963             0.7843994578286914            

dm2
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.830046532240195              0.807223203896048              0.8184757909456388            
Labeled
0.7942388654996676             0.7724001206740507             0.7831672784478237            

pas1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8608550160486239             0.8280562307035407             0.8441371459184357            
Labeled
0.8407088711329646             0.8086776588057545             

## CMU

In [291]:
cmu_dm = create_graph_conll(open('2014/submissions/cmu/open.dm.1.sdp'))
cmu_pas = create_graph_conll(open('2014/submissions/cmu/open.pas.1.sdp'))
cmu_pcedt = create_graph_conll(open('2014/submissions/cmu/open.pcedt.1.sdp'))

print('open')
print('\ndm')
dependencies(dm, cmu_dm)
print('\npas')
dependencies(pas, cmu_pas)
print('\npcedt')
dependencies(pcedt, cmu_pcedt)

open

dm
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8602947588732885             0.8503210791707969             0.855278843444524             
Labeled
0.8446411441527862             0.834848941947162              0.8397164965212303            

pas
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.9179046656560552             0.8949615713065756             0.9062879380020289            
Labeled
0.9078322385042951             0.8851409052092228             0.8963429844838767            

pcedt
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8884605087014725             0.8179442938131625             0.8517453798767968            
Labeled
0.7681392235609104             0.7071727877742174             0.

## copenhagen-malmo

In [292]:
copenhagen_malmo_dm1 = create_graph_conll(open('2014/submissions/copenhagen-malmö/closed.dm.1.sdp'))
copenhagen_malmo_dm2 = create_graph_conll(open('2014/submissions/copenhagen-malmö/closed.dm.2.sdp'))
copenhagen_malmo_pas1 = create_graph_conll(open('2014/submissions/copenhagen-malmö/closed.pas.1.sdp'))
copenhagen_malmo_pas2 = create_graph_conll(open('2014/submissions/copenhagen-malmö/closed.pas.2.sdp'))
copenhagen_malmo_pcedt1 = create_graph_conll(open('2014/submissions/copenhagen-malmö/closed.pcedt.1.sdp'))
copenhagen_malmo_pcedt2 = create_graph_conll(open('2014/submissions/copenhagen-malmö/closed.pcedt.2.sdp'))

print('closed')
print('\ndm1')
dependencies(dm, copenhagen_malmo_dm1)
print('\ndm2')
dependencies(dm, copenhagen_malmo_dm2)
print('\npas1')
dependencies(pas, copenhagen_malmo_pas1)
print('\npas2')
dependencies(pas, copenhagen_malmo_pas2)
print('\npcedt1')
dependencies(pcedt, copenhagen_malmo_pcedt1)
print('\npcedt2')
dependencies(pcedt, copenhagen_malmo_pcedt2)

closed

dm1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8916119962511715             0.8200232728526483             0.8543205441932513            
Labeled
0.8720243673851922             0.8020083609878033             0.8355521630783738            

dm2
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8676464194095396             0.8600181011076153             0.86381541924592              
Labeled
0.847819470411757              0.8403654699823299             0.8440760140253669            

pas1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.9260925686889175             0.8247717269920515             0.8725004777540348            
Labeled
0.9129633044440346             0.81307889377915               

## in-house

In [475]:
in_house_dm = create_graph_conll(open('2014/submissions/in-house/open.dm.1.sdp'))
in_house_pas = create_graph_conll(open('2014/submissions/in-house/open.pas.1.sdp'))
in_house_pcedt = create_graph_conll(open('2014/submissions/in-house/open.pcedt.1.sdp'))

print('open')
print('cmu')
print('\ndm')
dependencies(dm, in_house_dm)
print('\npas')
dependencies(pas, in_house_pas)
print('\npcedt')
dependencies(pcedt, in_house_pcedt)

open
cmu

dm
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.9361361966901439             0.9337154678274361             0.9349242653087645            
Labeled
0.9258090999438275             0.923415075636771              0.9246105381262677            

pas
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.9320273468314488             0.9313538724298759             0.9316904879250864            
Labeled
0.920884827767552              0.9202194048479275             0.9205519960571712            

pcedt
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.6556762005649718             0.7323145181168351             0.691879555669407             
Labeled
0.4089424435028249             0.45674143455755484          

## linkoping

In [294]:
linköping_dm = create_graph_conll(open('2014/submissions/linköping/closed.dm.1.sdp'))
linköping_pas = create_graph_conll(open('2014/submissions/linköping/closed.pas.1.sdp'))
linköping_pcedt = create_graph_conll(open('2014/submissions/linköping/closed.pcedt.1.sdp'))

print('closed')
print('cmu')
print('\ndm')
dependencies(dm, linköping_dm)
print('\npas')
dependencies(pas, linköping_pas)
print('\npcedt')
dependencies(pcedt, linköping_pcedt)

closed
cmu

dm
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8145385149201944             0.8093780976597854             0.8119501070062042            
Labeled
0.7854354614850798             0.7804594233504288             0.7829395360902743            

pas
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.7814455517663809             0.7752085659856796             0.778314564131313             
Labeled
0.7615799754991226             0.7555015437167444             0.7585285824992993            

pcedt
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.7366635687732342             0.7815134335716046             0.7584260255950245            
Labeled
0.6065520446096654             0.6434804042395859         

## Peking

In [295]:
peking_dm1 = create_graph_conll(open('2014/submissions/peking/closed.dm.1.sdp'))
peking_dm2 = create_graph_conll(open('2014/submissions/peking/closed.dm.2.sdp'))
peking_pas1 = create_graph_conll(open('2014/submissions/peking/closed.pas.1.sdp'))
peking_pas2 = create_graph_conll(open('2014/submissions/peking/closed.pas.2.sdp'))
peking_pcedt1 = create_graph_conll(open('2014/submissions/peking/closed.pcedt.1.sdp'))
peking_pcedt2 = create_graph_conll(open('2014/submissions/peking/closed.pcedt.2.sdp'))

print('closed')
print('cmu')
print('\ndm1')
dependencies(dm, peking_dm1)
print('\ndm2')
dependencies(dm, peking_dm2)
print('\npas1')
dependencies(pas, peking_pas1)
print('\npas2')
dependencies(pas, peking_pas2)
print('\npcedt1')
dependencies(pcedt, peking_pcedt1)
print('\npcedt2')
dependencies(pcedt, peking_pcedt2)

closed
cmu

dm1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.9170401617013797             0.8994526569840107             0.9081612671612889            
Labeled
0.9027155286053256             0.8854027496444425             0.8939753269075956            

dm2
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.9207344180670402             0.8926000948153257             0.906449001028514             
Labeled
0.9068640526362586             0.8791535577296039             0.892793837669869             

pas1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.9454803885072253             0.917624646915851              0.9313442786898908            
Labeled
0.9343801820704593             0.9068514747421664         

## Potsdam

In [307]:
# closed
potsdam_dm1_c = create_graph_conll(open('2014/submissions/potsdam/closed.dm.1.sdp'))
potsdam_dm2_c = create_graph_conll(open('2014/submissions/potsdam/closed.dm.2.sdp'))
potsdam_pas1_c = create_graph_conll(open('2014/submissions/potsdam/closed.pas.1.sdp'))
potsdam_pas2_c = create_graph_conll(open('2014/submissions/potsdam/closed.pas.2.sdp'))
potsdam_pcedt1_c = create_graph_conll(open('2014/submissions/potsdam/closed.pcedt.1.sdp'))
potsdam_pcedt2_c = create_graph_conll(open('2014/submissions/potsdam/closed.pcedt.2.sdp'))

# open
potsdam_dm1_o = create_graph_conll(open('2014/submissions/potsdam/open.dm.1.sdp'))
potsdam_dm2_o = create_graph_conll(open('2014/submissions/potsdam/open.dm.2.sdp'))
potsdam_pas1_o = create_graph_conll(open('2014/submissions/potsdam/open.pas.1.sdp'))
potsdam_pas2_o = create_graph_conll(open('2014/submissions/potsdam/open.pas.2.sdp'))
potsdam_pcedt1_o = create_graph_conll(open('2014/submissions/potsdam/open.pcedt.1.sdp'))
potsdam_pcedt2_o = create_graph_conll(open('2014/submissions/potsdam/open.pcedt.2.sdp'))


print('closed')
print('\ndm1')
dependencies(dm, potsdam_dm1_c)
print('\ndm2')
dependencies(dm, potsdam_dm2_c)
print('\npas1')
dependencies(pas, potsdam_pas1_c)
print('\npas2')
dependencies(pas, potsdam_pas2_c)
print('\npcedt1')
dependencies(pas, potsdam_pcedt1_c)
print('\npcedt2')
dependencies(pcedt, potsdam_pcedt2_c)

print('open')
print('\ndm1')
dependencies(dm, potsdam_dm1_o)
print('\ndm2')
dependencies(dm, potsdam_dm2_o)
print('\npas1')
dependencies(pas, potsdam_pas1_o)
print('\npas2')
dependencies(pas, potsdam_pas2_o)
print('\npcedt1')
dependencies(pas, potsdam_pcedt1_o)
print('\npcedt2')
dependencies(pcedt, potsdam_pcedt2_o)

closed

dm1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8546772523299966             0.7469723742619488             0.7972034405041165            
Labeled
0.8338675477094531             0.7287850708960049             0.7777931097925579            

dm2
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8155523944997629             0.8154118001982502             0.8154820912891686            
Labeled
0.7935686883055304             0.7934318838081282             0.7935002801603379            

pas1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8970254403131116             0.7527754056362084             0.81859418529895              
Labeled
0.881839530332681              0.7400315312356303             

## Turku

In [308]:
turku_dm1 = create_graph_conll(open('2014/submissions/turku/open.dm.1.sdp'))
turku_dm2 = create_graph_conll(open('2014/submissions/turku/open.dm.2.sdp'))
turku_pas1 = create_graph_conll(open('2014/submissions/turku/open.pas.1.sdp'))
turku_pas2 = create_graph_conll(open('2014/submissions/turku/open.pas.2.sdp'))
turku_pcedt1 = create_graph_conll(open('2014/submissions/turku/open.pcedt.1.sdp'))
turku_pcedt2 = create_graph_conll(open('2014/submissions/turku/open.pcedt.2.sdp'))

print('open')
print('\ndm1')
dependencies(dm, turku_dm1)
print('\ndm2')
dependencies(dm, turku_dm2)
print('\npas1')
dependencies(pas, turku_pas1)
print('\npas2')
dependencies(pas, turku_pas2)
print('\npcedt1')
dependencies(pcedt, turku_pcedt1)
print('\npcedt2')
dependencies(pcedt, turku_pcedt2)

open

dm1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8181818181818182             0.8444166702581563             0.8310922587486744            
Labeled
0.798930972564413              0.824548549756497              0.8115376458112407            

dm2
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8286830594130887             0.8409688402361764             0.8347807486631016            
Labeled
0.8093600033974604             0.8213593069861655             0.8153155080213904            

pas1
--------------------------------------
precision                      recall                         f-score                       
Unlabeled
0.8885191347753744             0.8945017407869671             0.8915004010016858            
Labeled
0.8741639750742227             0.8800499244564146             0.

# ---------------------------------------------------------------------
# Dependency by type

## Function: Dependency by type

In [495]:
def dependencies_by_type(types, gold, test, sort=False):
    """
    Precision, recall, F-score for dependencies
    """
    
    count = 0
    
    print('Dependencies by type')
    print('{:<18} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}'.
          format('Type', 'Gold', 'Test', 'Match', 'Precision', 'Recall', 'F-score'))
    
    saved = []
    
    for _type in types:
        
        found = 0
        gold_size = 0
        test_size = 0
            
        for a, b in zip(gold, test):
        
            # graphs
            _gold = a[0].edges(data='label')
            _test = b[0].edges(data='label')
        
            # sizes
            for edge in _gold:
                _current_type = edge[2]
                if _current_type == _type:
                    gold_size += 1
                    
                    
            for edge in _test:
                _current_type = edge[2]
                if _current_type == _type:
                    test_size += 1
                    
            for edge in _test:
                _current_type = edge[2]
                if edge in _gold and _current_type == _type:
                    found += 1
               
        p = 0 if test_size == 0 else found / test_size
        r = 0 if gold_size == 0 else found / gold_size
        if found > 0:
            f = 2 * ((p * r) / (p + r))
        else:
            f = 0
            
        if not sort and sort != 0:
            print('{:<18} {:<10} {:<10} {:<10} {:<10.4f} {:<10.4f} {:<10.4f}'.
              format(_type, gold_size, test_size, found, p, r, f))
        else:
            saved.append((_type, gold_size, test_size, found, p, r, f))
            
    if sort or sort == 0:
        sorted_list = sorted(saved, key=lambda x: x[sort], reverse=True)
        
        for el in sorted_list:
            print('{:<18} {:<10} {:<10} {:<10} {:<10.4f} {:<10.4f} {:<10.4f}'.
                  format(el[0], el[1], el[2], el[3], el[4], el[5], el[6]))

## Data for dependency types

## Alpage

In [513]:
sort = 1
print('\ndm1')
dependencies_by_type(dm_args, dm, alpage_dm1_c, sort)
print('\ndm2')
dependencies_by_type(dm_args, dm, alpage_dm2_c, sort)
print('\npas1')
dependencies_by_type(pas_args, pas, alpage_pas1_c, sort)
print('\npas2')
dependencies_by_type(pas_args, pas, alpage_pas2_c, sort)
print('\npcedt2')
dependencies_by_type(pcedt_args, pcedt, alpage_pcedt2_c, sort)

print('Alpage - open')
print('\ndm1')
dependencies_by_type(dm_args, dm, alpage_dm1_o, sort)
print('\ndm2')
dependencies_by_type(dm_args, dm, alpage_dm2_o, sort)
print('\npas1')
dependencies_by_type(pas_args, pas, alpage_pas1_o, sort)
print('\npas2')
dependencies_by_type(pas_args, pas, alpage_pas2_o, sort)
print('\npcedt2')
dependencies_by_type(pcedt_args, pcedt, alpage_pcedt2_o, sort)


dm1
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
ARG1               8812       8125       6323       0.7782     0.7175     0.7466    
ARG2               5374       5415       4256       0.7860     0.7920     0.7890    
compound           2514       2548       2332       0.9152     0.9276     0.9214    
BV                 2412       2439       2351       0.9639     0.9747     0.9693    
poss               533        527        498        0.9450     0.9343     0.9396    
_and_c             476        498        249        0.5000     0.5231     0.5113    
loc                321        295        170        0.5763     0.5296     0.5519    
ARG3               296        271        123        0.4539     0.4155     0.4339    
appos              206        195        139        0.7128     0.6748     0.6933    
mwe                189        185        175        0.9459     0.9259     0.9358    
times              163        160      

## CMU

In [512]:
sort = 6
print('open')
print('cmu')
print('\ndm')
dependencies_by_type(dm_args, dm, cmu_dm, sort)
print('\npas')
dependencies_by_type(pas_args, pas, cmu_pas, sort)
print('\npcedt')
dependencies_by_type(pcedt_args, pcedt, cmu_pcedt, sort)

open
cmu

dm
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
neg                124        124        124        1.0000     1.0000     1.0000    
plus               4          4          4          1.0000     1.0000     1.0000    
times              163        162        162        1.0000     0.9939     0.9969    
of                 32         31         31         1.0000     0.9688     0.9841    
BV                 2412       2439       2363       0.9688     0.9797     0.9742    
poss               533        534        511        0.9569     0.9587     0.9578    
compound           2514       2533       2312       0.9128     0.9196     0.9162    
_or_c              85         74         69         0.9324     0.8118     0.8679    
unspec_manner      4          3          3          1.0000     0.7500     0.8571    
mwe                189        176        156        0.8864     0.8254     0.8548    
ARG2               5374       5

## Copenhagen-malmo

In [301]:
sort = 6
print('closed')
print('\ndm')
dependencies_by_type(dm_args, dm, copenhagen_malmo_dm1, sort)
print('\ndm2')
dependencies_by_type(dm_args, dm, copenhagen_malmo_dm2, sort)
print('\npas')
dependencies_by_type(pas_args, pas, copenhagen_malmo_pas1, sort)
print('\npas2')
dependencies_by_type(pas_args, pas, copenhagen_malmo_pas2, sort)
print('\npcedt')
dependencies_by_type(pcedt_args, pcedt, copenhagen_malmo_pcedt1, sort)
print('\npcedt2')
dependencies_by_type(pcedt_args, pcedt, copenhagen_malmo_pcedt2, sort)

closed

dm
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
neg                124        124        124        1.0000     1.0000     1.0000    
times              163        163        163        1.0000     1.0000     1.0000    
plus               4          4          4          1.0000     1.0000     1.0000    
BV                 2412       2418       2363       0.9773     0.9797     0.9785    
poss               533        542        519        0.9576     0.9737     0.9656    
of                 32         31         30         0.9677     0.9375     0.9524    
compound           2514       2474       2310       0.9337     0.9189     0.9262    
unspec_manner      4          5          4          0.8000     1.0000     0.8889    
ARG2               5374       5443       4730       0.8690     0.8802     0.8745    
_or_c              85         77         68         0.8831     0.8000     0.8395    
ARG1               8812       845

## In-House

In [302]:
sort = 6
print('open')
print('cmu')
print('\ndm')
dependencies_by_type(dm_args, dm, in_house_dm, sort)
print('\npas')
dependencies_by_type(pas_args, pas, in_house_pas, sort)
print('\npcedt')
dependencies_by_type(pcedt_args, pcedt, in_house_pcedt, sort)

open
cmu

dm
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
neg                124        124        124        1.0000     1.0000     1.0000    
measure            13         13         13         1.0000     1.0000     1.0000    
of                 32         32         32         1.0000     1.0000     1.0000    
_but+not_c         1          1          1          1.0000     1.0000     1.0000    
_versus_c          1          1          1          1.0000     1.0000     1.0000    
plus               4          4          4          1.0000     1.0000     1.0000    
_though_c          1          1          1          1.0000     1.0000     1.0000    
times              163        160        160        1.0000     0.9816     0.9907    
BV                 2412       2416       2375       0.9830     0.9847     0.9838    
poss               533        531        520        0.9793     0.9756     0.9774    
compound           2514       2

## linköping

In [303]:
sort = 6
print('open')
print('cmu')
print('\ndm')
dependencies_by_type(dm_args, dm, linköping_dm, sort)
print('\npas')
dependencies_by_type(pas_args, pas, linköping_pas, sort)
print('\npcedt')
dependencies_by_type(pcedt_args, pcedt, linköping_pcedt, sort)

open
cmu

dm
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
plus               4          4          4          1.0000     1.0000     1.0000    
times              163        162        162        1.0000     0.9939     0.9969    
neg                124        126        124        0.9841     1.0000     0.9920    
of                 32         31         31         1.0000     0.9688     0.9841    
BV                 2412       2430       2347       0.9658     0.9731     0.9694    
poss               533        534        491        0.9195     0.9212     0.9203    
compound           2514       2458       2262       0.9203     0.8998     0.9099    
mwe                189        175        163        0.9314     0.8624     0.8956    
unspec_manner      4          5          4          0.8000     1.0000     0.8889    
ARG1               8812       8871       6680       0.7530     0.7581     0.7555    
ARG2               5374       5

## Peking

In [515]:
sort = 6
print('closed')
print('\ndm')
dependencies_by_type(dm_args, dm, peking_dm1, sort)
print('\ndm2')
dependencies_by_type(dm_args, dm, peking_dm2, sort)
print('\npas')
dependencies_by_type(pas_args, pas, peking_pas1, sort)
print('\npas2')
dependencies_by_type(pas_args, pas, peking_pas2, sort)
print('\npcedt')
dependencies_by_type(pcedt_args, pcedt, peking_pcedt1, sort)
print('\npcedt2')
dependencies_by_type(pcedt_args, pcedt, peking_pcedt2, sort)

closed

dm
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
neg                124        124        124        1.0000     1.0000     1.0000    
plus               4          4          4          1.0000     1.0000     1.0000    
unspec_manner      4          4          4          1.0000     1.0000     1.0000    
times              163        162        162        1.0000     0.9939     0.9969    
BV                 2412       2422       2375       0.9806     0.9847     0.9826    
poss               533        540        523        0.9685     0.9812     0.9748    
of                 32         32         31         0.9688     0.9688     0.9688    
compound           2514       2503       2374       0.9485     0.9443     0.9464    
mwe                189        183        175        0.9563     0.9259     0.9409    
ARG2               5374       5460       4863       0.8907     0.9049     0.8977    
ARG1               8812       868

## Potsdam

In [305]:
sort = 6
print('closed')
print('\ndm1')
dependencies_by_type(dm_args, dm, potsdam_dm1_c, sort)
print('\ndm2')
dependencies_by_type(dm_args, dm, potsdam_dm2_c, sort)
print('\npas1')
dependencies_by_type(pas_args, pas, potsdam_pas1_c, sort)
print('\npas2')
dependencies_by_type(pas_args, pas, potsdam_pas2_c, sort)
print('\npcedt1')
dependencies_by_type(pcedt_args, pcedt, potsdam_pcedt1_c, sort)
print('\npcedt2')
dependencies_by_type(pcedt_args, pcedt, potsdam_pcedt2_c, sort)

print('open')
print('\ndm1')
dependencies_by_type(dm_args, dm, potsdam_dm1_o, sort)
print('\ndm2')
dependencies_by_type(dm_args, dm, potsdam_dm2_o, sort)
print('\npas1')
dependencies_by_type(pas_args, pas, potsdam_pas1_o, sort)
print('\npas2')
dependencies_by_type(pas_args, pas, potsdam_pas2_o, sort)
print('\npcedt1')
dependencies_by_type(pcedt_args, pcedt, potsdam_pcedt1_o, sort)
print('\npcedt2')
dependencies_by_type(pcedt_args, pcedt, potsdam_pcedt2_o, sort)

closed

dm1
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
plus               4          4          4          1.0000     1.0000     1.0000    
neg                124        125        124        0.9920     1.0000     0.9960    
times              163        162        160        0.9877     0.9816     0.9846    
BV                 2412       2431       2357       0.9696     0.9772     0.9734    
of                 32         32         31         0.9688     0.9688     0.9688    
poss               533        503        474        0.9423     0.8893     0.9151    
compound           2514       2414       2220       0.9196     0.8831     0.9010    
unspec_manner      4          3          3          1.0000     0.7500     0.8571    
mwe                189        178        140        0.7865     0.7407     0.7629    
ARG1               8812       7679       6190       0.8061     0.7025     0.7507    
measure            13         11

## Turku

In [306]:
sort = 6
print('closed')
print('\ndm')
dependencies_by_type(dm_args, dm, turku_dm1, sort)
print('\ndm2')
dependencies_by_type(dm_args, dm, turku_dm2, sort)
print('\npas')
dependencies_by_type(pas_args, pas, turku_pas1, sort)
print('\npas2')
dependencies_by_type(pas_args, pas, turku_pas2, sort)
print('\npcedt')
dependencies_by_type(pcedt_args, pcedt, turku_pcedt1, sort)
print('\npcedt2')
dependencies_by_type(pcedt_args, pcedt, turku_pcedt2, sort)

closed

dm
Dependencies by type
Type               Gold       Test       Match      Precision  Recall     F-score   
plus               4          4          4          1.0000     1.0000     1.0000    
neg                124        126        124        0.9841     1.0000     0.9920    
times              163        160        160        1.0000     0.9816     0.9907    
of                 32         31         31         1.0000     0.9688     0.9841    
BV                 2412       2441       2350       0.9627     0.9743     0.9685    
poss               533        523        498        0.9522     0.9343     0.9432    
compound           2514       2524       2296       0.9097     0.9133     0.9115    
unspec_manner      4          3          3          1.0000     0.7500     0.8571    
measure            13         11         10         0.9091     0.7692     0.8333    
ARG2               5374       5718       4406       0.7705     0.8199     0.7944    
_or_c              85         78 

# ------------------------------------------------------
# Dependency by distance

## Function: Dependency by distance

In [381]:
def dependencies_by_length(gold, test, arg=False):
    """
    Precision, recall, F-score for dependencies
    """
    
    longest_g = 0
    longest_t = 0
    
    if arg:
        print('Dependencies by length and specific type')
        print('{:<8} {:<18} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}'.
              format('Length', 'Type', 'Gold', 'Test', 'Match', 'Precision', 'Recall', 'F-score'))
    else:
        print('Dependencies by length')
        print('{:<8} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}'.
              format('Length', 'Gold', 'Test', 'Match', 'Precision', 'Recall', 'F-score'))

    for length in range(0, 100):
        found = 0
        gold_size = 0
        test_size = 0
        
        for a, b in zip(gold, test):

            # graphs
            _gold = a[0].edges(data='label')
            _test = b[0].edges(data='label')

            # counting sizes
            for edge in _gold:
                start_end = sorted([edge[0], edge[1]])
                _len = start_end[1] - start_end[0]
                if _len > longest_g:
                    longest_g = _len
                if _len == length:
                    gold_size += 1

            for edge in _test:
                start_end = sorted([edge[0], edge[1]])
                _len = start_end[1] - start_end[0]
                if _len > longest_t:
                    longest_t = _len
                if _len == length:
                    if edge in _gold:
                        found += 1
                    test_size += 1

        p = 0 if test_size == 0 else found / test_size
        r = 0 if gold_size == 0 else found / gold_size
        if found > 0:
            f = 2 * ((p * r) / (p + r))
        else:
            f = 0
            
        if gold_size > 0:
            if arg:
                print('{:<8} {:<18} {:<10} {:<10} {:<10} {:<10.4f} {:<10.4f} {:<10.4f}'.
                      format(length, arg, gold_size, test_size, found, p, r, f))
            else:
                print('{:<8} {:<10} {:<10} {:<10} {:<10.4f} {:<10.4f} {:<10.4f}'.
                      format(length, gold_size, test_size, found, p, r, f))

    print('Longest dependency in gold: {}'.format(longest_g))
    print('Longest dependency in test: {}'.format(longest_t))

## Alpage

In [384]:
print('closed')
print('\ndm1')
dependencies_by_length(dm, alpage_dm1_c)
print('\ndm2')
dependencies_by_length(dm, alpage_dm2_c)
print('\npas1')
dependencies_by_length(pas, alpage_pas1_c)
print('\npas2')
dependencies_by_length(pas, alpage_pas2_c)
print('\npcedt2')
dependencies_by_length(pcedt, alpage_pcedt2_c)

print('open')
print('\ndm1')
dependencies_by_length(dm, alpage_dm1_o)
print('\ndm2')
dependencies_by_length(dm, alpage_dm2_o)
print('\npas1')
dependencies_by_length(pas, alpage_pas1_o)
print('\npas2')
dependencies_by_length(pas, alpage_pas2_o)
print('\npcedt2')
dependencies_by_length(pcedt, alpage_pcedt2_o)

closed

dm1
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       1319       896        0.6793     0.6657     0.6724    
1        10044      10173      9126       0.8971     0.9086     0.9028    
2        5063       4969       4251       0.8555     0.8396     0.8475    
3        2586       2438       1888       0.7744     0.7301     0.7516    
4        1240       1160       771        0.6647     0.6218     0.6425    
5        735        669        382        0.5710     0.5197     0.5442    
6        505        411        211        0.5134     0.4178     0.4607    
7        297        286        108        0.3776     0.3636     0.3705    
8        266        236        91         0.3856     0.3421     0.3625    
9        185        173        48         0.2775     0.2595     0.2682    
10       186        136        34         0.2500     0.1828     0.2112    
11       146        104        32         0.3077     0.2192     0

## CMU

In [382]:
print('open')
print('\ndm')
dependencies_by_length(dm, cmu_dm)
print('\npas')
dependencies_by_length(pas, cmu_pas)
print('\npcedt')
dependencies_by_length(pcedt, cmu_pcedt)

open

dm
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       1348       933        0.6921     0.6932     0.6927    
1        10044      10526      9412       0.8942     0.9371     0.9151    
2        5063       5284       4589       0.8685     0.9064     0.8870    
3        2586       2735       2215       0.8099     0.8565     0.8326    
4        1240       1250       945        0.7560     0.7621     0.7590    
5        735        676        495        0.7322     0.6735     0.7016    
6        505        404        299        0.7401     0.5921     0.6579    
7        297        257        168        0.6537     0.5657     0.6065    
8        266        185        134        0.7243     0.5038     0.5942    
9        185        149        94         0.6309     0.5081     0.5629    
10       186        120        87         0.7250     0.4677     0.5686    
11       146        0          0          0.0000     0.0000     0.00

## Copenhagen-malmo

In [385]:
print('closed')
print('\ndm1')
dependencies_by_length(dm, copenhagen_malmo_dm1)
print('\ndm2')
dependencies_by_length(dm, copenhagen_malmo_dm2)
print('\npas1')
dependencies_by_length(pas, copenhagen_malmo_pas1)
print('\npas2')
dependencies_by_length(pas, copenhagen_malmo_pas2)
print('\npcedt1')
dependencies_by_length(pcedt, copenhagen_malmo_pcedt1)
print('\npcedt2')
dependencies_by_length(pcedt, copenhagen_malmo_pcedt2)

closed

dm1
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       0          0          0.0000     0.0000     0.0000    
1        10044      10126      9302       0.9186     0.9261     0.9224    
2        5063       4991       4515       0.9046     0.8918     0.8981    
3        2586       2502       2134       0.8529     0.8252     0.8388    
4        1240       1133       917        0.8094     0.7395     0.7729    
5        735        677        502        0.7415     0.6830     0.7110    
6        505        413        311        0.7530     0.6158     0.6776    
7        297        297        186        0.6263     0.6263     0.6263    
8        266        227        152        0.6696     0.5714     0.6166    
9        185        186        110        0.5914     0.5946     0.5930    
10       186        166        109        0.6566     0.5860     0.6193    
11       146        119        70         0.5882     0.4795     0

## In-House

In [386]:
print('open')
print('cmu')
print('\ndm')
dependencies_by_length(dm, in_house_dm)
print('\npas')
dependencies_by_length(pas, in_house_pas)
print('\npcedt')
dependencies_by_length(pcedt, in_house_pcedt)

open
cmu

dm
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       1344       1266       0.9420     0.9406     0.9413    
1        10044      10037      9571       0.9536     0.9529     0.9532    
2        5063       5015       4779       0.9529     0.9439     0.9484    
3        2586       2589       2350       0.9077     0.9087     0.9082    
4        1240       1207       1079       0.8940     0.8702     0.8819    
5        735        746        624        0.8365     0.8490     0.8427    
6        505        488        411        0.8422     0.8139     0.8278    
7        297        298        231        0.7752     0.7778     0.7765    
8        266        253        215        0.8498     0.8083     0.8285    
9        185        205        153        0.7463     0.8270     0.7846    
10       186        182        159        0.8736     0.8548     0.8641    
11       146        143        117        0.8182     0.8014     

## Linkoping

In [387]:
print('closed')
print('cmu')
print('\ndm')
dependencies_by_length(dm, linköping_dm)
print('\npas')
dependencies_by_length(pas, linköping_pas)
print('\npcedt')
dependencies_by_length(pcedt, linköping_pcedt)

closed
cmu

dm
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       1469       918        0.6249     0.6820     0.6522    
1        10044      10188      9057       0.8890     0.9017     0.8953    
2        5063       5097       4241       0.8321     0.8376     0.8348    
3        2586       2619       1932       0.7377     0.7471     0.7424    
4        1240       1235       788        0.6381     0.6355     0.6368    
5        735        720        398        0.5528     0.5415     0.5471    
6        505        380        212        0.5579     0.4198     0.4791    
7        297        291        130        0.4467     0.4377     0.4422    
8        266        221        111        0.5023     0.4173     0.4559    
9        185        169        58         0.3432     0.3135     0.3277    
10       186        156        60         0.3846     0.3226     0.3509    
11       146        116        45         0.3879     0.3082   

## Peking

In [388]:
print('closed')
print('cmu')
print('\ndm1')
dependencies_by_length(dm, peking_dm1)
print('\ndm2')
dependencies_by_length(dm, peking_dm2)
print('\npas1')
dependencies_by_length(pas, peking_pas1)
print('\npas2')
dependencies_by_length(pas, peking_pas2)
print('\npcedt1')
dependencies_by_length(pcedt, peking_pcedt1)
print('\npcedt2')
dependencies_by_length(pcedt, peking_pcedt2)

closed
cmu

dm1
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       1331       1217       0.9144     0.9042     0.9092    
1        10044      10284      9560       0.9296     0.9518     0.9406    
2        5063       5140       4716       0.9175     0.9315     0.9244    
3        2586       2598       2264       0.8714     0.8755     0.8735    
4        1240       1179       999        0.8473     0.8056     0.8260    
5        735        660        542        0.8212     0.7374     0.7771    
6        505        410        336        0.8195     0.6653     0.7344    
7        297        244        188        0.7705     0.6330     0.6950    
8        266        192        162        0.8438     0.6090     0.7074    
9        185        146        108        0.7397     0.5838     0.6526    
10       186        132        106        0.8030     0.5699     0.6667    
11       146        92         72         0.7826     0.4932  

## Potsdam

In [389]:
print('closed')
print('\ndm1')
dependencies_by_length(dm, potsdam_dm1_c)
print('\ndm2')
dependencies_by_length(dm, potsdam_dm2_c)
print('\npas1')
dependencies_by_length(pas, potsdam_pas1_c)
print('\npas2')
dependencies_by_length(pas, potsdam_pas2_c)
print('\npcedt1')
dependencies_by_length(pas, potsdam_pcedt1_c)
print('\npcedt2')
dependencies_by_length(pcedt, potsdam_pcedt2_c)

print('open')
print('\ndm1')
dependencies_by_length(dm, potsdam_dm1_o)
print('\ndm2')
dependencies_by_length(dm, potsdam_dm2_o)
print('\npas1')
dependencies_by_length(pas, potsdam_pas1_o)
print('\npas2')
dependencies_by_length(pas, potsdam_pas2_o)
print('\npcedt1')
dependencies_by_length(pas, potsdam_pcedt1_o)
print('\npcedt2')
dependencies_by_length(pcedt, potsdam_pcedt2_o)

closed

dm1
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       1186       896        0.7555     0.6657     0.7077    
1        10044      10062      9023       0.8967     0.8983     0.8975    
2        5063       4391       3816       0.8691     0.7537     0.8073    
3        2586       1992       1585       0.7957     0.6129     0.6924    
4        1240       817        595        0.7283     0.4798     0.5785    
5        735        461        306        0.6638     0.4163     0.5117    
6        505        295        185        0.6271     0.3663     0.4625    
7        297        227        104        0.4581     0.3502     0.3969    
8        266        172        81         0.4709     0.3045     0.3699    
9        185        128        56         0.4375     0.3027     0.3578    
10       186        95         51         0.5368     0.2742     0.3630    
11       146        88         42         0.4773     0.2877     0

## Turku

In [390]:
print('open')
print('\ndm1')
dependencies_by_length(dm, turku_dm1)
print('\ndm2')
dependencies_by_length(dm, turku_dm2)
print('\npas1')
dependencies_by_length(pas, turku_pas1)
print('\npas2')
dependencies_by_length(pas, turku_pas2)
print('\npcedt1')
dependencies_by_length(pcedt, turku_pcedt1)
print('\npcedt2')
dependencies_by_length(pcedt, turku_pcedt2)

open

dm1
Dependencies by length
Length   Gold       Test       Match      Precision  Recall     F-score   
0        1346       1348       1073       0.7960     0.7972     0.7966    
1        10044      10913      9241       0.8468     0.9201     0.8819    
2        5063       5324       4426       0.8313     0.8742     0.8522    
3        2586       2609       2022       0.7750     0.7819     0.7784    
4        1240       1188       838        0.7054     0.6758     0.6903    
5        735        662        436        0.6586     0.5932     0.6242    
6        505        407        270        0.6634     0.5347     0.5921    
7        297        269        160        0.5948     0.5387     0.5654    
8        266        207        123        0.5942     0.4624     0.5201    
9        185        177        99         0.5593     0.5351     0.5470    
10       186        147        87         0.5918     0.4677     0.5225    
11       146        113        70         0.6195     0.4795     0.5

# ++++++++++++++++++++++++++++++++++++++
# Dangling nodes

## Function: compare dangling nodes

In [517]:
def dangling_nodes(gold, test, arg=False):
    
    gold_size = 0
    test_size = 0
    found = 0
    
    count = 0
    
    for a, b in zip(gold, test):
        
        if count > 10:
            break
        count += 1
        
        # graph data
        _gold_graph = a[0].edges(data='label')
        _gold_data = a[1]
        _gold_visited = []
        
        _test_graph = b[0].edges(data='label')
        _test_data = b[1]
        _test_visited = []
        
        # appending in-degree
        for edge in _gold_graph:
            if edge[1] not in _gold_visited:
                _gold_visited.append(edge[1])
        
        for edge in _test_graph:
            if edge[1] not in _test_visited:
                _test_visited.append(edge[1])
            
        # finding dangling nodes
        for k, v in _gold_data.items():
            if k not in _gold_visited:
                gold_size += 1
                
        for k, v in _test_data.items():
            if k not in _test_visited:
                test_size += 1
                if k not in _gold_visited:
                    found += 1
                
    p = found / test_size
    r = found / gold_size
    f = 2 * ((p * r) / (p + r))
        
    
    print('{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}'.
                      format('Gold', 'Test', 'Match', 'Precision', 'Recall', 'F-score'))
    print('{:<10} {:<10} {:<10} {:<10.4f} {:<10.4f} {:<10.4f}'.
                      format(gold_size, test_size, found, p, r, f))
    
dangling_nodes(dm, alpage_dm1_c)

Gold       Test       Match      Precision  Recall     F-score   
107        114        100        0.8772     0.9346     0.9050    


## alpage

In [504]:
print('closed')
print('\ndm1')
dangling_nodes(dm, alpage_dm1_c)
print('\ndm2')
dangling_nodes(dm, alpage_dm2_c)
print('\npas1')
dangling_nodes(pas, alpage_pas1_c)
print('\npas2')
dangling_nodes(pas, alpage_pas2_c)
print('\npcedt2')
dangling_nodes(pcedt, alpage_pcedt2_c)

print('open')
print('\ndm1')
dangling_nodes(dm, alpage_dm1_o)
print('\ndm2')
dangling_nodes(dm, alpage_dm2_o)
print('\npas1')
dangling_nodes(pas, alpage_pas1_o)
print('\npas2')
dangling_nodes(pas, alpage_pas2_o)
print('\npcedt2')
dangling_nodes(pcedt, alpage_pcedt2_o)

closed

dm1
Gold       Test       Match      Precision  Recall     F-score   
13633      13767      12868      0.9347     0.9439     0.9393    

dm2
Gold       Test       Match      Precision  Recall     F-score   
13633      13689      12903      0.9426     0.9465     0.9445    

pas1
Gold       Test       Match      Precision  Recall     F-score   
8732       9254       8465       0.9147     0.9694     0.9413    

pas2
Gold       Test       Match      Precision  Recall     F-score   
8732       9102       8563       0.9408     0.9806     0.9603    

pcedt2
Gold       Test       Match      Precision  Recall     F-score   
19889      19684      19243      0.9776     0.9675     0.9725    
open

dm1
Gold       Test       Match      Precision  Recall     F-score   
13633      13793      13000      0.9425     0.9536     0.9480    

dm2
Gold       Test       Match      Precision  Recall     F-score   
13633      14559      13104      0.9001     0.9612     0.9296    

pas1
Gold       Test   

In [508]:
print('open')
print('\ndm')
dangling_nodes(dm, cmu_dm)
print('\npas')
dangling_nodes(pas, cmu_pas)
print('\npcedt')
dangling_nodes(pcedt, cmu_pcedt)

open

dm
Gold       Test       Match      Precision  Recall     F-score   
13633      13673      12870      0.9413     0.9440     0.9426    

pas
Gold       Test       Match      Precision  Recall     F-score   
8732       9049       8618       0.9524     0.9869     0.9693    

pcedt
Gold       Test       Match      Precision  Recall     F-score   
19889      19833      19383      0.9773     0.9746     0.9759    
